## left open for future research : 
<p>is composition and data encapsulation (and other internal quality attribute) affected by Extract Class refactoring ??</p>
<h4>Data Encapsulation metric </h4>
<li>CCDA </li>
<li>CIDA </li>
<li>COA </li>
<li>DAM </li>
<h4>Composition metric </h4>
<li>MOA</li>
<li>CPCC </li>


In [2]:
%matplotlib inline

#matrix math
import numpy as np

#data manipulation
import pandas as pd
#matrix data structure
from patsy import dmatrices
#for error logging
import warnings

import matplotlib.pyplot as plt

COLOR = 'white'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR
plt.rcParams['xtick.color'] = COLOR
plt.rcParams['ytick.color'] = COLOR
plt.rcParams['figure.figsize'] = (8, 6)
plt.rcParams['font.size'] = 14

#outputs probability between 0 and 1, used to help define our logistic regression curve
def sigmoid(x):
    '''Sigmoid function of x.'''
    return 1/(1+np.exp(-x))
#makes the random numbers predictable
#(pseudo-)random numbers work by starting with a number (the seed), 
#multiplying it by a large number, then taking modulo of that product. 
#The resulting number is then used as the seed to generate the next "random" number. 
#When you set the seed (every time), it does the same thing every time, giving you the same numbers.
#good for reproducing results for debugging
np.random.seed(0) # set the seed

##Step 1 - Define model parameters (hyperparameters)

## algorithm settings
#the minimum threshold for the difference between the predicted output and the actual output
#this tells our model when to stop learning, when our prediction capability is good enough
tol=1e-8 # convergence tolerance
#how long to train for?
max_iter = 20 # maximum allowed iterations

In [14]:
## data creation settings
#Covariance measures how two variables move together. 
#It measures whether the two move in the same direction (a positive covariance) 
#or in opposite directions (a negative covariance). 
covariance_x_z = 0.95 # covariance between x and z

n = 1000 # number of observations (size of dataset to generate) 
sigma = 1 # variance of noise - how spread out is the data?

## model settings
beta_x, beta_z, beta_v = -4, .9, 1 # true beta coefficients
var_x, var_z, var_v = 1, 1, 4 # variances of inputs

## Step 2 - Generate and organize our data

#The multivariate normal, multinormal or Gaussian distribution is a generalization of the one-dimensional normal 
#distribution to higher dimensions. Such a distribution is specified by its mean and covariance matrix.
#so we generate values input values - (x, v, z) using normal distributions

#a probability distribution is a function that provides us the probabilities of all 
#possible outcomes of a stochastic process. 



#lets keep x and z closely related (height and weight) with multivariate normal
x, z = np.random.multivariate_normal([0,0], [[var_x,covariance_x_z],[covariance_x_z,var_z]], n).T

In [71]:
#blood presure
v = np.random.normal(0,var_v,n)**3
#create a pandas dataframe (easily parseable object for manipulation)
df = pd.DataFrame({'x' : x, 'z' : z, 'v' : v})

In [19]:
#compute the log odds for our 3 independent variables
#using the sigmoid function 
df['log_odds'] = sigmoid(df[['x','z','v']].dot([beta_x,beta_z,beta_v]) + sigma*np.random.normal(0,1,n))

#compute the probability sample from binomial distribution
#a binomial random variable is the number of successes x has in n repeated trials of a binomial experiment. 
#The probability distribution of a binomial random variable is called a binomial distribution. 
df['y'] = [np.random.binomial(1,p) for p in df.log_odds]

#create a dataframe that encompasses our input data, model formula, and outputs
## the model specification you want to fit
formula = 'y ~ x + z + v + np.exp(x) + I(v**2 + z)'
y, X = dmatrices(formula, df, return_type='dataframe')
#print it
X.head(3)

,Intercept,x,z,v,np.exp(x),I(v ** 2 + z)
0,1.0,-2.346006,-1.922000,351.329306,0.095751,123430.359545
1,1.0,1.018850,1.129388,4.100831,2.770008,17.946205
2,1.0,-0.397505,0.265339,60.003876,0.671995,3600.730457


In [21]:
y.head(5)

,y
0,1.0
1,0.0
2,1.0
3,0.0
4,0.0


<a name="algorithms"></a>
<hr>
### Algorithm Setup

We begin with a quick function for catching singular matrix errors that we will use to decorate our Newton steps.

In [5]:
#like dividing by zero (Wtff omgggggg universe collapses)
def catch_singularity(f):
    '''Silences LinAlg Errors and throws a warning instead.'''
    
    def silencer(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except np.linalg.LinAlgError:
            warnings.warn('Algorithm terminated - singular Hessian!')
            return args[0]
    return silencer


### Explanation of a Single Newton Step

Recall that Newton's method for maximizing / minimizing a given function $f(\beta)$ iteratively computes the following estimate:

$$
\beta^+ = \beta - Hf(\beta)^{-1}\nabla f(\beta)
$$

The Hessian of the log-likelihood for logistic regression is given by:

hessian of our function = negative tranpose of (N times (p+1) times (N x N diagional matrix of weights, each is p*(1-p) times X again


$$
Hf(\beta) = -X^TWX
$$
and the gradient is:

gradient of our function = tranpose of X times (column vector - N vector of probabilities)

$$
\nabla f(\beta) = X^T(y-p)
$$
where
$$
W := \text{diag}\left(p(1-p)\right)
$$
and $p$ are the predicted probabilites computed at the current value of $\beta$.

### Connection to Iteratively Reweighted Least Squares (IRLS)
*For logistic regression, this step is actually equivalent to computing a weighted least squares estimator at each iteration!* 
The method of least squares is about estimating
parameters by minimizing the squared discrepancies
between observed data, on the one hand, and their
expected values on the other

I.e.,
$$
\beta^+ = \arg\min_\beta (z-X\beta)^TW(z-X\beta)
$$
with $W$ as before and the *adjusted response* $z$ is given by
$$
z := X\beta + W^{-1}(y-p)
$$

**Takeaway:** This is fun, but in fact it can be leveraged to derive asymptotics and inferential statistics about the computed MLE $\beta^*$!

### Our implementations
Below we implement a single step of Newton's method, and we compute $Hf(\beta)^{-1}\nabla f(\beta)$ using `np.linalg.lstsq(df,b)` to solve the equation $Ax = b$.  Note that this does not require us to compute the actual inverse of the Hessian.

In [6]:
@catch_singularity
def newton_step(curr, X, lam=None):
    '''One naive step of Newton's Method'''
    
    #how to compute inverse? http://www.mathwarehouse.com/algebra/matrix/images/square-matrix/inverse-matrix.gif
    
    ## compute necessary objects
    #create probability matrix, miniminum 2 dimensions, tranpose (flip it)
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    #create weight matrix from it
    W = np.diag((p*(1-p))[:,0])
    #derive the hessian 
    hessian = X.T.dot(W).dot(X)
    #derive the gradient
    grad = X.T.dot(y-p)
    
    ## regularization step (avoiding overfitting)
    if lam:
        # Return the least-squares solution to a linear matrix equation
        step, *_ = np.linalg.lstsq(hessian + lam*np.eye(curr.shape[0]), grad)
    else:
        step, *_ = np.linalg.lstsq(hessian, grad)
        
    ## update our 
    beta = curr + step
    
    return beta

Next, we implement Newton's method in a *slightly* different way; this time, at each iteration, we actually compute the full inverse of the Hessian using `np.linalg.inv()`.

In [7]:
@catch_singularity
def alt_newton_step(curr, X, lam=None):
    '''One naive step of Newton's Method'''
    
    ## compute necessary objects
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    W = np.diag((p*(1-p))[:,0])
    hessian = X.T.dot(W).dot(X)
    grad = X.T.dot(y-p)
    
    ## regularization
    if lam:
        #Compute the inverse of a matrix.
        step = np.dot(np.linalg.inv(hessian + lam*np.eye(curr.shape[0])), grad)
    else:
        step = np.dot(np.linalg.inv(hessian), grad)
        
    ## update our weights
    beta = curr + step
    
    return beta

<a name="conv"></a>
<hr>
### Convergence Setup

First we implement coefficient convergence; this stopping criterion results in convergence whenever
$$
\|\beta^+ - \beta\|_\infty < \epsilon
$$
where $\epsilon$ is a given tolerance.

In [8]:
def check_coefs_convergence(beta_old, beta_new, tol, iters):
    '''Checks whether the coefficients have converged in the l-infinity norm.
    Returns True if they have converged, False otherwise.'''
    #calculate the change in the coefficients
    coef_change = np.abs(beta_old - beta_new)
    
    #if change hasn't reached the threshold and we have more iterations to go, keep training
    return not (np.any(coef_change>tol) & (iters < max_iter))

<a name="numerics"></a>
<hr>
## Numerical Examples

### Standard Newton with Coefficient Convergence

In [9]:
## initial conditions
#initial coefficients (weight values), 2 copies, we'll update one
beta_old, beta = np.ones((len(X.columns),1)), np.zeros((len(X.columns),1))

#num iterations we've done so far
iter_count = 0
#have we reached convergence?
coefs_converged = False

#if we haven't reached convergence... (training step)
while not coefs_converged:
    
    #set the old coefficients to our current
    beta_old = beta
    #perform a single step of newton's optimization on our data, set our updated beta values
    beta = newton_step(beta, X, lam=lam)
    #increment the number of iterations
    iter_count += 1
    
    #check for convergence between our old and new beta values
    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)
    
print('Iterations : {}'.format(iter_count))
print('Beta : {}'.format(beta))

Iterations : 18
Beta : [[ 26320895.67739986]
 [ -6345540.54736841]
 [-11270652.29286714]
 [ -9577183.80803834]
 [ 20672070.05807817]
 [ -7783568.04197949]]


/home/riansyah/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
